In [1]:
!pip install gdown

import gdown

url = 'https://drive.google.com/uc?id=1yxP8E8BGcQXqisWB6Dy2Z8wuD2-qxown'

gdown.download(url, 'archivo.zip', quiet=False)

import zipfile
import os

# Ruta del archivo ZIP en Google Drive
zip_file_path = '/kaggle/working/archivo.zip'

# Ruta donde deseas extraer los archivos
extract_path = '/content/extracted_files'

# Extraer el contenido del archivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Listar los archivos extraídos
os.listdir(extract_path)

Downloading...
From (original): https://drive.google.com/uc?id=1yxP8E8BGcQXqisWB6Dy2Z8wuD2-qxown
From (redirected): https://drive.google.com/uc?id=1yxP8E8BGcQXqisWB6Dy2Z8wuD2-qxown&confirm=t&uuid=19d5f9bc-9282-4936-b5d8-6e93b70c3466
To: /kaggle/working/archivo.zip
100%|██████████| 2.18G/2.18G [00:33<00:00, 64.9MB/s]


['train.csv', 'test', 'train', 'sample.csv']

In [2]:
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split
import cv2


2024-07-02 19:36:00.478737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 19:36:00.478849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 19:36:00.616549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_df = pd.read_csv('/content/extracted_files/train.csv')
train_df['filename'] = train_df['ID'] + '_' + train_df['location'] + '.jpg'
train_df['level'] = train_df['level'].apply(lambda x: '0' if x == 0 else '1')

train_df.head()

,ID,location,level,filename
0,d3d9446802a44259755d38e6d163e820,left,0,d3d9446802a44259755d38e6d163e820_left.jpg
1,d3d9446802a44259755d38e6d163e820,right,0,d3d9446802a44259755d38e6d163e820_right.jpg
2,c51ce410c124a10e0db5e4b97fc2af39,left,0,c51ce410c124a10e0db5e4b97fc2af39_left.jpg
3,c51ce410c124a10e0db5e4b97fc2af39,right,0,c51ce410c124a10e0db5e4b97fc2af39_right.jpg
4,9bf31c7ff062936a96d3c8bd1f8f2ff3,left,1,9bf31c7ff062936a96d3c8bd1f8f2ff3_left.jpg


In [4]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
#import os

In [5]:
# Definir parámetros de imagen y entrenamiento
img_height, img_width = 224, 224
batch_size = 32
num_classes = 2

# Dividir en conjunto de entrenamiento y validación
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['level'], random_state=42)

# Generadores de datos
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rotation_range=40,width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/extracted_files/train/train',
    x_col='filename',
    y_col='level',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='/content/extracted_files/train/train',
    x_col='filename',
    y_col='level',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 19670 validated image filenames belonging to 2 classes.
Found 4918 validated image filenames belonging to 2 classes.


In [6]:
# Construir el modelo
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

#agregandole regularizacion l2 para evitar sobreajuste
from tensorflow.keras.regularizers import L2

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=L2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min'),
    EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, mode='min', min_lr=1e-7)
]


In [8]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
#Balancear clases
y_train = train_df['level'].values

# Calcular los pesos de las clases
class_weights = compute_class_weight(class_weight="balanced",classes=np.unique(y_train),y=y_train)

class_weight_dict = dict(enumerate(class_weights))

In [9]:
# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size,
    class_weight=class_weight_dict,
    callbacks=callbacks
)

# Guardar el modelo
model.save('retinopathy_detection.keras')

Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/614 ━━━━━━━━━━━━━━━━━━━━ 4:10:24 25s/step - accuracy: 0.5000 - loss: 14.5230

I0000 00:00:1719949058.816392     122 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719949058.871718     122 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


529/614 ━━━━━━━━━━━━━━━━━━━━ 43s 515ms/step - accuracy: 0.4879 - loss: 8.1750

...

614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.4900 - loss: 7.6894

W0000 00:00:1719949375.196470     122 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


614/614 ━━━━━━━━━━━━━━━━━━━━ 359s 546ms/step - accuracy: 0.4900 - loss: 7.6841 - val_accuracy: 0.7337 - val_loss: 1.8457 - learning_rate: 1.0000e-04
Epoch 2/25
  1/614 ━━━━━━━━━━━━━━━━━━━━ 45s 74ms/step - accuracy: 0.4688 - loss: 1.4782

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
W0000 00:00:1719949395.945635     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4688 - loss: 1.4782 - val_accuracy: 0.9091 - val_loss: 1.1440 - learning_rate: 1.0000e-04
Epoch 3/25
614/614 ━━━━━━━━━━━━━━━━━━━━ 320s 515ms/step - accuracy: 0.4909 - loss: 1.3781 - val_accuracy: 0.2655 - val_loss: 5.1130 - learning_rate: 1.0000e-04
Epoch 4/25
614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 131us/step - accuracy: 0.5625 - loss: 0.9957 - val_accuracy: 0.2727 - val_loss: 4.3972 - learning_rate: 1.0000e-04
Epoch 5/25
614/614 ━━━━━━━━━━━━━━━━━━━━ 316s 509ms/step - accuracy: 0.5164 - loss: 1.0865 - val_accuracy: 0.2904 - val_loss: 1.0961 - learning_rate: 1.0000e-04
Epoch 6/25
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4688 - loss: 0.9977 - val_accuracy: 0.7273 - val_loss: 0.8305 - learning_rate: 1.0000e-04
Epoch 7/25
614/614 ━━━━━━━━━━━━━━━━━━━━ 319s 514ms/step - accuracy: 0.5089 - loss: 0.9652 - val_accuracy: 0.6150 - val_loss: 0.8469 - learning_rate: 1.0000e-04
Epoch 8/25
614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 132us/step - accur

In [10]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Fine tunning
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size,
    callbacks=callbacks
)

# Guardar el modelo
model.save('retinopathy_detection_finetunning.keras')

Epoch 1/10
  2/614 ━━━━━━━━━━━━━━━━━━━━ 1:04 106ms/step - accuracy: 0.4453 - loss: 0.8759 

W0000 00:00:1719952752.882384     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


573/614 ━━━━━━━━━━━━━━━━━━━━ 19s 476ms/step - accuracy: 0.5229 - loss: 0.8234

W0000 00:00:1719953025.248040     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step - accuracy: 0.5248 - loss: 0.8219

W0000 00:00:1719953045.677468     122 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


614/614 ━━━━━━━━━━━━━━━━━━━━ 330s 504ms/step - accuracy: 0.5248 - loss: 0.8219 - val_accuracy: 0.6612 - val_loss: 0.7696 - learning_rate: 1.0000e-05
Epoch 2/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5312 - loss: 0.7448 - val_accuracy: 0.6364 - val_loss: 0.7617 - learning_rate: 1.0000e-05
Epoch 3/10


W0000 00:00:1719953063.902185     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


614/614 ━━━━━━━━━━━━━━━━━━━━ 296s 477ms/step - accuracy: 0.6324 - loss: 0.7540 - val_accuracy: 0.6985 - val_loss: 0.7150 - learning_rate: 1.0000e-05
Epoch 4/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 127us/step - accuracy: 0.7500 - loss: 0.6967 - val_accuracy: 0.8636 - val_loss: 0.6178 - learning_rate: 1.0000e-05
Epoch 5/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 292s 471ms/step - accuracy: 0.7100 - loss: 0.7149 - val_accuracy: 0.7239 - val_loss: 0.6727 - learning_rate: 1.0000e-05
Epoch 6/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 127us/step - accuracy: 0.6250 - loss: 0.7820 - val_accuracy: 0.6818 - val_loss: 0.7422 - learning_rate: 1.0000e-05
Epoch 7/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 299s 481ms/step - accuracy: 0.7231 - loss: 0.6915 - val_accuracy: 0.7341 - val_loss: 0.6725 - learning_rate: 1.0000e-05
Epoch 8/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 0s 128us/step - accuracy: 0.7188 - loss: 0.6577 - val_accuracy: 0.8182 - val_loss: 0.6008 - learning_rate: 1.0000e-05
Epoch 9/10
614/614 ━━━━━━━━━━━━━━━━━━━━ 303s 489ms/step -

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Generador para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/extracted_files/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 10538 images belonging to 1 classes.


In [21]:
# Predicciones
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
# Extraer puntajes flotantes
scores = predictions[:, 0]

330/330 ━━━━━━━━━━━━━━━━━━━━ 37s 112ms/step


In [22]:
import os
import pandas as pd

filenames = [os.path.splitext(os.path.basename(f))[0] for f in test_generator.filenames]

# Verificar la longitud de ambas listas
assert len(filenames) == len(scores), "Las longitudes de filenames y scores no coinciden"

# Crear DataFrame de resultados
submission_df = pd.DataFrame({
    'ID': filenames,
    'score': scores
})

submission_df.to_csv('submissionDR.csv', index=False)

print(submission_df.head())


                                       ID     score
0   0002ac0d783338cfeab0b2bdbd872cda_left  0.468029
1  0002ac0d783338cfeab0b2bdbd872cda_right  0.496906
2   000e82a96e908e73372bed4e5cb98096_left  0.460893
3  000e82a96e908e73372bed4e5cb98096_right  0.482462
4   0012a83c1e6cda9d73a22dbeaac47e8f_left  0.486997
